In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json
from pprint import pprint

# Import API key
from config import gkey

In [3]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

city_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sao filipe,14.90,-24.50,78.98,66,80,7.20,CV,1605848346
1,bay roberts,47.60,-53.26,28.00,58,90,20.80,CA,1605848346
2,skjervoy,70.03,20.97,33.80,69,75,28.86,NO,1605848150
3,vestmannaeyjar,63.44,-20.27,44.22,67,92,15.61,IS,1605848346
4,vardo,70.37,31.11,35.60,80,40,21.92,NO,1605848346


In [54]:
# Humidity Heatmap

# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

figure = gmaps.figure()


locations = city_data[["Latitude","Longitude"]]
humidity = city_data["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
figure.add_layer(heat_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
ideal_temperature = city_data[(city_data["Max Temp"] < 80) & (city_data["Max Temp"] > 70)]
# Wind speed less than 10 mph.
ideal_wind = ideal_temperature[(ideal_temperature["Wind Speed"] < 10)]
# Zero cloudiness
ideal_city_weather = ideal_wind[(ideal_wind["Cloudiness"] == 0)]
ideal_city_weather



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,mahanoro,-19.90,48.80,76.89,62,0,7.52,MG,1605848352
97,neuquen,-38.95,-68.06,75.20,41,0,3.36,AR,1605848368
120,lazaro cardenas,17.96,-102.20,76.30,80,0,5.86,MX,1605848373
122,coahuayana,18.73,-103.68,76.08,70,0,3.42,MX,1605848374
235,khanpur,28.65,70.66,72.41,22,0,9.08,PK,1605848399
332,pokhara,28.23,83.98,72.01,26,0,5.41,NP,1605848455
351,salalah,17.02,54.09,78.80,44,0,2.24,OM,1605848434
421,saint-pierre,-21.34,55.48,77.00,57,0,8.05,RE,1605848480
425,sikar,27.62,75.15,71.35,22,0,6.06,IN,1605848481
500,chor,25.52,69.77,77.76,18,0,6.67,PK,1605848500


## Hotel Map

In [17]:
target_coordinates = ideal_city_weather[["Latitude","Longitude"]]
target_search = "hotel"

# Set parameters to search for hotels with 5000 meters.
target_radius = 5000
target_type = "lodging"
hotel_df = []

params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Hit the Google Places API for each city's coordinates.
for index,row in ideal_city_weather.iterrows():
    try:
        lat = row["Latitude"]
        lng = row["Longitude"]

        params["location"] = f"{lat},{lng}"

        # run a request using our params dictionary
        response = requests.get(url, params=params)

        # convert response to json
        hotel_data = response.json()

         # Store the first Hotel result into the DataFrame.
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.append(hotel_name)
    except:
        hotel_df.append("No hotels found")



In [19]:
ideal_city_weather["Hotel Name"] = hotel_df

/Users/sherman/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
ideal_city_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,mahanoro,-19.90,48.80,76.89,62,0,7.52,MG,1605848352,Bungalow le Prestige
97,neuquen,-38.95,-68.06,75.20,41,0,3.36,AR,1605848368,Hotel Leloir
120,lazaro cardenas,17.96,-102.20,76.30,80,0,5.86,MX,1605848373,Hotel de Casablanca
122,coahuayana,18.73,-103.68,76.08,70,0,3.42,MX,1605848374,Hotel Los Arcos
235,khanpur,28.65,70.66,72.41,22,0,9.08,PK,1605848399,Shakir Hotel & Bar BQ
332,pokhara,28.23,83.98,72.01,26,0,5.41,NP,1605848455,Hotel Pristine Himalaya
351,salalah,17.02,54.09,78.80,44,0,2.24,OM,1605848434,Al Dyafa Hotel Suites
421,saint-pierre,-21.34,55.48,77.00,57,0,8.05,RE,1605848480,Le Battant Des Lames
425,sikar,27.62,75.15,71.35,22,0,6.06,IN,1605848481,Hotel Raisina Hill
500,chor,25.52,69.77,77.76,18,0,6.67,PK,1605848500,No hotels found


In [60]:
# Dropping city with no hotels found
city_hotels = ideal_city_weather.drop(index=500)
city_hotels

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,mahanoro,-19.90,48.80,76.89,62,0,7.52,MG,1605848352,Bungalow le Prestige
97,neuquen,-38.95,-68.06,75.20,41,0,3.36,AR,1605848368,Hotel Leloir
120,lazaro cardenas,17.96,-102.20,76.30,80,0,5.86,MX,1605848373,Hotel de Casablanca
122,coahuayana,18.73,-103.68,76.08,70,0,3.42,MX,1605848374,Hotel Los Arcos
235,khanpur,28.65,70.66,72.41,22,0,9.08,PK,1605848399,Shakir Hotel & Bar BQ
332,pokhara,28.23,83.98,72.01,26,0,5.41,NP,1605848455,Hotel Pristine Himalaya
351,salalah,17.02,54.09,78.80,44,0,2.24,OM,1605848434,Al Dyafa Hotel Suites
421,saint-pierre,-21.34,55.48,77.00,57,0,8.05,RE,1605848480,Le Battant Des Lames
425,sikar,27.62,75.15,71.35,22,0,6.06,IN,1605848481,Hotel Raisina Hill
547,puerto carreno,6.19,-67.49,75.97,85,0,8.43,CO,1605848511,Hotel Carreño Plaza


## Plot markers on top of the heatmap

In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_hotels.iterrows()]
locations = city_hotels[["Latitude", "Longitude"]]


In [64]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map

figure.add_layer(markers)
# Display figure

figure

Figure(layout=FigureLayout(height='420px'))